In [1]:
#v2.0
import tkinter as tk
from tkinter import filedialog
import pandas as pd
from docx import Document
from ollama import chat
import threading
import random

# --------------------------
# Guidelines + Blogs
# --------------------------
guidelines = {
    "HIPAA Privacy Rule – Minimum Necessary Standard": "https://www.hhs.gov/sites/default/files/hipaa-privacy-rule.pdf",
    "Privacy Rule: 45 CFR §164.500 – §164.534": "https://www.ecfr.gov/current/title-45/subtitle-A/subchapter-C/part-164/subpart-E",
    "Security Rule: 45 CFR §164.302 – §164.318": "https://www.ecfr.gov/current/title-45/subtitle-A/subchapter-C/part-164/subpart/C",
    "Breach Notification Rule: 45 CFR §164.400 – §164.414": "https://www.ecfr.gov/current/title-45/subtitle-A/subchapter-C/part-164/subpart/D",
    "HITECH Act": "https://www.hhs.gov/sites/default/files/hitech-act-text.pdf",
    "NIST SP 800-122": "https://nvlpubs.nist.gov/nistpubs/Legacy/SP/nistspecialpublication800-122.pdf",
    "NIST SP 800-53 Rev. 5": "https://nvlpubs.nist.gov/nistpubs/SpecialPublications/NIST.SP.800-53r5.pdf",
    "NIST SP 800-66 Rev.1": "https://nvlpubs.nist.gov/nistpubs/Legacy/SP/nistspecialpublication800-66r1.pdf",
    "NIST Privacy Framework": "https://www.nist.gov/system/files/documents/2020/01/16/NIST%20Privacy%20Framework_V1.0.pdf",
    "NIST Risk Management Framework (SP 800-37)": "https://nvlpubs.nist.gov/nistpubs/SpecialPublications/NIST.SP.800-37r2.pdf",
    "HHS De-identification Guidelines": "https://www.hhs.gov/hipaa/for-professionals/privacy/special-topics/de-identification/index.html",
    "OCR Cybersecurity Guidance for Healthcare": "https://www.hhs.gov/sites/default/files/ocr-cybersecurity-newsletter-2021.pdf",
    "GDPR": "https://gdpr-info.eu/",
    "SLFLA Blog": "https://www.slflalaw.com/blog",
    "Strauss Law Blog": "https://www.strausslawblog.com"
}

# --------------------------
# Templates
# --------------------------
mitigation_templates_guidelines = [
    "Develop a comprehensive compliance plan aligned with HIPAA/NIST standards.",
    "Perform formal risk assessments and gap analysis per applicable regulations.",
    "Ensure PHI access logs are regularly reviewed and audited.",
    "Update policies and procedures to reflect latest regulatory guidance.",
    "Implement technical controls like encryption and secure backups."
]

prevention_templates_guidelines = [
    "Train staff on HIPAA minimum necessary standards and NIST privacy controls.",
    "Conduct periodic compliance audits with documented remediation plans.",
    "Establish a formal incident response team and reporting workflow.",
    "Deploy automated monitoring systems for unauthorized PHI access.",
    "Integrate privacy-by-design into organizational processes."
]

creative_modifiers_guidelines = [
    "Use numbered steps", "Reference official acts", "Focus on legal compliance",
    "Be highly structured", "Include guideline links explicitly"
]

# --------------------------
# LLM Insights with Structure
# --------------------------
def generate_mistral_insights(entity, desc):
    shuffled = list(guidelines.items())
    random.shuffle(shuffled)
    guidelines_text = "\n".join([f"- {k}: {v}" for k, v in shuffled])

    instruction = f"""
Mitigation: {random.choice(mitigation_templates_guidelines)}
Prevention: {random.choice(prevention_templates_guidelines)}
Extra instruction: {random.choice(creative_modifiers_guidelines)}
Map each recommendation to the relevant guideline and indicate possible violations.
Be sure to output the following in your response:
- Mitigation (actions to reduce risks)
- Prevention (actions to avoid risks)
- Resources (relevant links to guidelines and documents)
- Guidelines Violated (possible breaches or non-compliance issues)
"""
    sys_prompt = "You are a healthcare compliance advisor."
    user_prompt = f"""
Covered Entity: {entity}
Description: {desc[:500]}

{instruction}

Reference these guidelines:
{guidelines_text}
"""
    try:
        response = chat(
            model="mistral",
            messages=[
                {"role": "system", "content": sys_prompt},
                {"role": "user", "content": user_prompt}
            ]
        )
        # Assuming the response includes the sections Mitigation, Prevention, Resources, and Violations
        return response.message.content.strip()
    except Exception as e:
        return f"LLM Error: {e}"

# --------------------------
# Process Excel with Structured Output
# --------------------------
def analyze_file_mistral(path, result_text, status_label, analyze_button):
    try:
        df = pd.read_excel(path)
    except Exception as e:
        status_label.config(text=f"Error reading file: {e}")
        analyze_button.config(state=tk.NORMAL)
        return

    doc = Document()
    total = len(df)

    for idx, row in df.iterrows():
        entity = str(row.get("Name of Covered Entity", "Unknown Entity"))
        desc = str(row.get("Web Description", "No description provided"))

        status_label.config(text=f"Processing record {idx+1}/{total}...")
        insights = generate_mistral_insights(entity, desc)

        # GUI output with the structured format
        text_entry = f"\n--- Record {idx+1} ---\nEntity: {entity}\nDescription: {desc}\nInsights:\n{insights}\n{'-'*60}\n"
        result_text.insert(tk.END, text_entry)
        result_text.see(tk.END)

        # Word output with the structured format
        doc.add_heading(f"Record {idx+1}: {entity}", level=2)
        doc.add_paragraph(f"Description: {desc}")
        doc.add_paragraph(insights)

    doc.save("mistral_analysis.docx")
    status_label.config(text=" Analysis Complete (Saved to mistral_analysis.docx)")
    analyze_button.config(state=tk.NORMAL)


# --------------------------
# GUI
# --------------------------
def launch_guidelines_gui():
    root = tk.Tk()
    root.title("Healthcare Breach Analyzer – Guidelines Version")
    root.geometry("900x700")

    frame = tk.Frame(root, padx=10, pady=10)
    frame.pack(fill="x")
    tk.Label(frame, text="Upload an Excel file for analysis:").pack(side="left")

    output_label = tk.Label(root, text="", fg="blue")
    output_label.pack(fill="x", padx=10, pady=5)

    result_text = tk.Text(root, wrap="word", font=("Consolas", 10))
    result_text.pack(fill="both", expand=True, padx=10, pady=10)
    scrollbar = tk.Scrollbar(root, command=result_text.yview)
    scrollbar.pack(side="right", fill="y")
    result_text.config(yscrollcommand=scrollbar.set)

    analyze_button = tk.Button(frame, text="Upload Excel & Analyze", width=30)
    analyze_button.pack(side="left", padx=5)

    def on_upload():
        path = filedialog.askopenfilename(filetypes=[("Excel files", "*.xlsx")])
        if path:
            analyze_button.config(state=tk.DISABLED)
            threading.Thread(target=analyze_file_mistral, args=(path, result_text, output_label, analyze_button), daemon=True).start()

    analyze_button.config(command=on_upload)
    root.mainloop()

if __name__ == "__main__":
    launch_guidelines_gui()


Exception in thread Thread-5 (analyze_file_mistral):
Traceback (most recent call last):
  File "C:\Users\shsud\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "C:\Users\shsud\AppData\Local\Programs\Python\Python312\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Users\shsud\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\shsud\AppData\Local\Temp\ipykernel_29920\342820986.py", line 120, in analyze_file_mistral
  File "C:\Users\shsud\AppData\Local\Programs\Python\Python312\Lib\tkinter\__init__.py", line 3831, in insert
    self.tk.call((self._w, 'insert', index, chars) + args)
RuntimeError: main thread is not in main loop
